# Day Two: Location Enrichment

This notebook enriches BBF Location__c records with additional fields from ES Address__c.

## Day 1 Fields (Already Migrated)
- Name, City__c, State__c, County__c, PostalCode__c
- Street__c, Full_Address__c, CLLICode__c
- businessUnit__c, OwnerId, ES_Legacy_ID__c

## Day 2 Enrichment Fields
| BBF Field | ES Source | Transform |
|-----------|-----------|----------|
| Loc__c | Geocode_Lat_Long__c | Geolocation |
| StateCode__c | State__c | Direct copy |
| old_SiteId__c | Site_ID__c | Direct copy |
| Common_Name__c | Organization__c | Direct copy |
| Address_Validated_By__c | Verification_Used__c | Picklist map |
| Address_API_Message__c | Address_Return_Code__c | Direct copy |
| Market__c | Dimension_4_Market__c | Direct copy |
| Legacy_CLLI_Code__c | CLLI__c | Direct copy |
| strName__c | Thoroughfare_Name__c | Direct copy |
| Unique_Key__c | Unique_Constraint_Check__c | Direct copy |

In [ ]:
# === SETUP & IMPORTS ===

import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
from transformers.location_transformers import TRANSFORMERS, FIELD_MAPPING, apply_transformers

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("Loaded location transformers")
print(f"  - {len(TRANSFORMERS)} transformer functions")
print(f"  - {len(FIELD_MAPPING)} field mappings")
print("\n✅ Setup complete")

In [ ]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# BBF Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"

# Enrichment Options
DRY_RUN = True  # Set to False to actually update records
TEST_LIMIT = 100  # Set to None for all records

# Picklist value mapping for Address_Validated_By__c
VERIFICATION_TO_VALIDATED_BY = {
    'SmartyStreets': 'SmartyStreets',
    'Google Maps': 'Google',
    'UPS': None,  # No direct mapping
}

# Output file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"location_enrichment_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   DRY_RUN: {DRY_RUN}")
print(f"   TEST_LIMIT: {TEST_LIMIT}")
print(f"   Output: {output_file}")

if DRY_RUN:
    print("\n⚠️  DRY RUN MODE - No changes will be made")

In [ ]:
# === CONNECT TO SALESFORCE ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

In [ ]:
# === QUERY MIGRATED RECORDS ===

print("\n" + "=" * 80)
print("QUERYING MIGRATED RECORDS")
print("=" * 80)

# Query ES Address__c with enrichment fields
es_query = """
SELECT Id, BBF_New_Id__c, Name,
       Geocode_Lat_Long__c, Geocode_Lat_Long__Latitude__s, Geocode_Lat_Long__Longitude__s,
       Site_ID__c, Organization__c,
       Verification_Used__c, Address_Return_Code__c,
       Dimension_4_Market__c, CLLI__c, State__c,
       Thoroughfare_Name__c, Unique_Constraint_Check__c
FROM Address__c
WHERE BBF_New_Id__c != null
"""

if TEST_LIMIT:
    es_query += f" LIMIT {TEST_LIMIT}"

print("\n📌 Querying ES Address__c (enrichment fields)...")
es_result = es_sf.query_all(es_query)
es_records = es_result['records']
print(f"   Found {len(es_records)} migrated Address__c records")

# Build lookup by BBF_New_Id__c
es_lookup = {r['BBF_New_Id__c']: r for r in es_records}

# Query BBF Location__c current values
bbf_ids = list(es_lookup.keys())
bbf_records = []

print("\n📌 Querying BBF Location__c (current values)...")
chunk_size = 200
for i in range(0, len(bbf_ids), chunk_size):
    chunk = bbf_ids[i:i+chunk_size]
    ids_str = "','".join(chunk)
    
    bbf_query = f"""
    SELECT Id, ES_Legacy_ID__c, Name,
           Loc__c, Loc__Latitude__s, Loc__Longitude__s,
           old_SiteId__c, Common_Name__c,
           Address_Validated_By__c, Address_API_Message__c,
           Market__c, Legacy_CLLI_Code__c, StateCode__c,
           strName__c, Unique_Key__c
    FROM Location__c
    WHERE Id IN ('{ids_str}')
    """
    
    result = bbf_sf.query_all(bbf_query)
    bbf_records.extend(result['records'])

print(f"   Found {len(bbf_records)} Location__c records in BBF")

In [ ]:
# === BUILD ENRICHMENT UPDATES ===

print("\n" + "=" * 80)
print("BUILDING ENRICHMENT UPDATES")
print("=" * 80)

# Define enrichment field mappings (BBF -> ES)
ENRICHMENT_MAPPING = {
    'StateCode__c': 'State__c',
    'old_SiteId__c': 'Site_ID__c',
    'Common_Name__c': 'Organization__c',
    'Address_Validated_By__c': 'Verification_Used__c',
    'Address_API_Message__c': 'Address_Return_Code__c',
    'Legacy_CLLI_Code__c': 'CLLI__c',
    'strName__c': 'Thoroughfare_Name__c',
    'Unique_Key__c': 'Unique_Constraint_Check__c',
}

updates = []
update_details = []  # For Excel output
field_stats = {field: {'enriched': 0, 'already_set': 0, 'no_source': 0} for field in ENRICHMENT_MAPPING}
field_stats['Loc__c'] = {'enriched': 0, 'already_set': 0, 'no_source': 0}

for bbf_rec in bbf_records:
    bbf_id = bbf_rec['Id']
    
    # Get corresponding ES record
    if bbf_id not in es_lookup:
        continue
    es_rec = es_lookup[bbf_id]
    
    update_rec = {'Id': bbf_id}
    rec_details = {'bbf_id': bbf_id, 'name': bbf_rec.get('Name', 'N/A'), 'fields': []}
    
    # Process regular fields
    for bbf_field, es_field in ENRICHMENT_MAPPING.items():
        bbf_value = bbf_rec.get(bbf_field)
        es_value = es_rec.get(es_field)
        
        if bbf_value:
            field_stats[bbf_field]['already_set'] += 1
        elif es_value:
            # Apply transformation if needed
            if bbf_field == 'Address_Validated_By__c':
                transformed = VERIFICATION_TO_VALIDATED_BY.get(es_value)
                if transformed:
                    update_rec[bbf_field] = transformed
                    rec_details['fields'].append({
                        'field': bbf_field,
                        'old': bbf_value,
                        'new': transformed,
                        'source': es_value
                    })
                    field_stats[bbf_field]['enriched'] += 1
            else:
                update_rec[bbf_field] = es_value
                rec_details['fields'].append({
                    'field': bbf_field,
                    'old': bbf_value,
                    'new': es_value,
                    'source': es_value
                })
                field_stats[bbf_field]['enriched'] += 1
        else:
            field_stats[bbf_field]['no_source'] += 1
    
    # Process geolocation (special handling)
    bbf_lat = bbf_rec.get('Loc__Latitude__s')
    bbf_lng = bbf_rec.get('Loc__Longitude__s')
    es_lat = es_rec.get('Geocode_Lat_Long__Latitude__s')
    es_lng = es_rec.get('Geocode_Lat_Long__Longitude__s')
    
    if bbf_lat and bbf_lng:
        field_stats['Loc__c']['already_set'] += 1
    elif es_lat and es_lng:
        update_rec['Loc__c'] = {'latitude': es_lat, 'longitude': es_lng}
        rec_details['fields'].append({
            'field': 'Loc__c',
            'old': f"({bbf_lat}, {bbf_lng})",
            'new': f"({es_lat}, {es_lng})",
            'source': 'Geocode_Lat_Long__c'
        })
        field_stats['Loc__c']['enriched'] += 1
    else:
        field_stats['Loc__c']['no_source'] += 1
    
    # Only add if there are fields to update
    if len(update_rec) > 1:  # More than just Id
        updates.append(update_rec)
        update_details.append(rec_details)

print(f"\n📊 Enrichment Analysis:")
print(f"   Total BBF records analyzed: {len(bbf_records)}")
print(f"   Records needing updates: {len(updates)}")
print(f"\n   Field Statistics:")
print(f"   {'Field':<30} | {'Enriched':>10} | {'Already Set':>12} | {'No Source':>10}")
print(f"   {'-'*70}")
for field, stats in field_stats.items():
    print(f"   {field:<30} | {stats['enriched']:>10} | {stats['already_set']:>12} | {stats['no_source']:>10}")

In [ ]:
# === APPLY ENRICHMENT UPDATES ===

print("\n" + "=" * 80)
print("APPLYING ENRICHMENT UPDATES")
print("=" * 80)

if len(updates) == 0:
    print("\n⚠️  No updates to apply")
    update_results = []
elif DRY_RUN:
    print(f"\n🔍 DRY RUN - Would update {len(updates)} Location__c records")
    print("\nSample updates (first 5):")
    for i, detail in enumerate(update_details[:5], 1):
        print(f"\n{i}. {detail['name']} ({detail['bbf_id'][:15]}...)")
        for f in detail['fields'][:3]:
            print(f"   {f['field']}: {f['old']} -> {f['new']}")
    update_results = [{'success': True, 'id': u['Id']} for u in updates]  # Mock results
else:
    print(f"\n📌 Updating {len(updates)} Location__c records...")
    
    try:
        update_results = bbf_sf.bulk.Location__c.update(updates)
        
        success_count = sum(1 for r in update_results if r['success'])
        error_count = sum(1 for r in update_results if not r['success'])
        
        print(f"\n✅ Successfully updated: {success_count}")
        print(f"❌ Failed to update: {error_count}")
        
        if error_count > 0:
            print("\nFirst 5 errors:")
            error_count = 0
            for i, r in enumerate(update_results):
                if not r['success'] and error_count < 5:
                    print(f"  - {updates[i]['Id']}: {r['errors']}")
                    error_count += 1
    except Exception as e:
        print(f"\n❌ Error during update: {e}")
        update_results = []

In [ ]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()

# --- Sheet 1: Summary ---
ws1 = wb.active
ws1.title = "Summary"
ws1.append(["Location Enrichment Summary"])
ws1["A1"].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE UPDATE"])
ws1.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws1.append(["Records Analyzed:", len(bbf_records)])
ws1.append(["Records Updated:", len(updates)])
ws1.append([])
ws1.append(["Field", "Enriched", "Already Set", "No Source"])
for field, stats in field_stats.items():
    ws1.append([field, stats['enriched'], stats['already_set'], stats['no_source']])

# --- Sheet 2: Update Details ---
ws2 = wb.create_sheet("Update Details")
headers = ["BBF Location ID", "Name", "Field", "Old Value", "New Value", "ES Source"]
ws2.append(headers)

header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
for col, header in enumerate(headers, 1):
    cell = ws2.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill

for detail in update_details:
    for f in detail['fields']:
        ws2.append([
            detail['bbf_id'],
            detail['name'],
            f['field'],
            str(f['old']) if f['old'] else '',
            str(f['new']),
            f['source']
        ])

# Adjust column widths
for col in ws2.columns:
    max_length = max(len(str(cell.value or '')) for cell in col)
    ws2.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws2.freeze_panes = "A2"

# Save
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")

In [ ]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("LOCATION ENRICHMENT COMPLETE")
print("=" * 80)
print(f"\nRecords analyzed: {len(bbf_records)}")
print(f"Records {'would be ' if DRY_RUN else ''}updated: {len(updates)}")
print(f"Output file: {output_file}")

if DRY_RUN:
    print("\n⚠️  This was a DRY RUN - no changes were made")
    print("   Set DRY_RUN = False to apply updates")